## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [5]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
import networkx as nx
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False
from __future__ import print_function

In [6]:
import yaml

In [7]:
from IPython.html.widgets import FloatProgress
from IPython.display import display

In [8]:
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

In [9]:
saut="\n"

### Préparation des matrices de traits

In [10]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

In [11]:
sampleNumber="01-N"
genDigraphe=False
genGraphe=False
#samplePrefix="MGC-150916-extend-%s-paradigmes"%sampleNumber
#samplePrefix="MGC-160104-%s-VerbesActions-SILVER"%sampleNumber
#samplePrefix="MGC-160104-%s-ext3-paradigmes"%sampleNumber
samplePrefix="MGC-160104-%s-ext3-derivations-SILVER"%sampleNumber
#samplePrefix="MGC-160104-derivation2-%s-SILVER"%sampleNumber
#samplePrefix="MGC-160104-derivation2-derives2"
#samplePrefix="MGC-160104-%s-basesDerives"%sampleNumber
sampleFile=samplePrefix+".csv"
#goldFile="MGC-160104-01-N-VerbesActions-GOLD.csv"
#goldFile="MGC-160104-01-N-Gold.csv"
#analysisPrefix="MGC-150916-extend-%s"%sampleNumber
#analysisPrefix="MGC-160104-%s-VerbesActions-SILVER"%sampleNumber
#analysisPrefix="MGC-160104-%s-ext3-paradigmes"%sampleNumber
analysisPrefix="MGC-160104-%s-ext3-derivations-SILVER"%sampleNumber
#analysisPrefix="MGC-160104-derivation2-%s-SILVER"%sampleNumber
logfile_name=analysisPrefix+samplePrefix+".log"
logfile = codecs.open("2015-Data/"+logfile_name,mode='w',encoding="utf8")

In [12]:
paradigmes=pd.read_csv("2015-Data/"+sampleFile,sep=";",encoding="utf8")
del paradigmes[u"Unnamed: 0"]
paradigmes=paradigmes.dropna(axis=1,how='all')

In [13]:
phonologicalMap=analysisPrefix[-2:]
if debug: print(phonologicalMap)
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="-N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="-S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("-X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [14]:
cliqueFile="2015-Data/"+analysisPrefix+'-Network.pkl'
#paradigmes


In [15]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

In [16]:
goldCases=paradigmes.columns.tolist()
goldCases.remove("lexeme")
#goldCases

- sampleCases pour la liste des cases effectivement représentées dans le corpus de départ 

In [17]:
sampleCases=paradigmes.columns.values.tolist()
sampleCases.remove(u"lexeme")
#sampleCases

### Définition des objets pour la gestion des règles

In [18]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            print ("le numéro de forme n'est pas dans [1,2]",n,file=logfile)

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if idClasseForme in self.nbClasse:
                nTotal=self.nbClasse[idClasseForme]
                for patron in self.classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(self.classe[idClasseForme][patron])/nTotal
            else:
                if debug:
                    print (forme, file=logfile)
                    print ("pas de classe",idClasseForme, file=logfile)
                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
        else:
            print (forme, file=logfile) 
            print ("pas de patron", file=logfile)
        return sortieForme
        

### Comparer les cases analysées avec l'ensemble de toutes les cases

In [19]:
with open("2015-Data/"+analysisPrefix+'-Regles.pkl', 'rb') as input:
    resultatsLecture = pickle.load(input)

In [20]:
analyseCases=list(set([case for (case,autre) in resultatsLecture.keys() if autre in ["FP"]]))
print (sorted(analyseCases))
#    analyseCases=sampleCases

[u'FP', u'FS', u'ai1P', u'ai1S', u'ai2P', u'ai2S', u'ai3P', u'ai3S', u'fi1P', u'fi1S', u'fi2P', u'fi2S', u'fi3P', u'fi3S', u'ii1P', u'ii1S', u'ii2P', u'ii2S', u'ii3P', u'ii3S', u'inf', u'is1P', u'is1S', u'is3P', u'is3S', u'pI1P', u'pI2P', u'pI2S', u'pP', u'pc1P', u'pc1S', u'pc2P', u'pc2S', u'pc3P', u'pc3S', u'pi1P', u'pi1S', u'pi2P', u'pi2S', u'pi3P', u'pi3S', u'ppFP', u'ppFS', u'ppMP', u'ppMS', u'ps1P', u'ps1S', u'ps2P', u'ps2S', u'ps3P', u'ps3S']


#### Calculer le score de la clique

In [21]:
def cliqueScore(clique,graph):
    score=0
    for (depart,arrivee) in it.combinations_with_replacement(clique,2):
        score+=graph[depart][arrivee]["weight"]
    return score

## Génération des formes

In [22]:
basesDerives=pd.read_csv("2015-Data/MGC-160104-01-N-basesDerives.csv",sep=";",encoding="utf8")

In [23]:
with open("2015-Data/"+analysisPrefix+'-Network.pkl', 'rb') as input:
    infoCliques=pickle.load(input)
cliques=infoCliques["cliques"]
cliquesScores=infoCliques["cliquesScores"]
cliquesListes=infoCliques["cliquesListes"]

In [24]:
#cliques

In [25]:
listeTest=list(set(case[0].split("-")[0] for case in cliques))
len(listeTest)

195

In [26]:
analyseCases=list(set(case.split("-")[-1] for clique in cliques for case in clique ))
#sorted(analyseCases)

In [27]:
longueurCliques={}
for clique in cliques:
    longueur=len(clique)
    if not longueur in longueurCliques:
        longueurCliques[longueur]=1
    else:
        longueurCliques[longueur]+=1
longueurCliques

{2: 448,
 3: 796,
 4: 923,
 5: 1437,
 6: 1942,
 7: 2101,
 8: 1982,
 9: 1918,
 10: 2179,
 11: 2022,
 12: 1847,
 13: 1732,
 14: 1747,
 15: 1500,
 16: 1480,
 17: 1600,
 18: 1605,
 19: 1761,
 20: 1501,
 21: 1504,
 22: 1932,
 23: 2204,
 24: 1721,
 25: 1129,
 26: 773,
 27: 574,
 28: 578,
 29: 463,
 30: 254,
 31: 209,
 32: 133,
 33: 45,
 34: 24,
 35: 26,
 36: 30,
 37: 12,
 38: 1,
 39: 9,
 40: 2,
 44: 1}

### Faire la liste des cases lexicalisées de l'échantillon

In [28]:
nbFormesLexicales={}
casesLexicales={element:paradigmes[paradigmes["lexeme"]==element].columns[paradigmes[paradigmes["lexeme"]==element].notnull().iloc[0]].tolist() for element in listeTest}
for element in casesLexicales:
    casesLexicales[element].remove("lexeme")
    nbFormesLexicales[element]=len(casesLexicales[element])
#nbFormesLexicales

In [29]:
def cliques2Tableau(cliques):
    paradigmesListe=[]
    for n,clique in enumerate(cliques):
        paradigmeClique={}
        for element in clique:
            elements=element.split("-")
            lexeme=elements[0]
            nbInitial=len(casesLexicales[lexeme])
            forme=elements[-2]
            taminfo=elements[-1]
            paradigmeClique[taminfo]=forme
        paradigmeClique["lexeme"]=lexeme
        paradigmeClique["score"]=cliquesScores[n]
        paradigmeClique["ajouts"]=len(clique)-nbInitial
        paradigmesListe.append(paradigmeClique)
    return pd.DataFrame(paradigmesListe,columns=["lexeme"]+analyseCases+[u"score",u"ajouts"])


In [30]:
cliquesPANDAS=cliques2Tableau(cliques)

In [31]:
filtreDERIVE=~(cliquesPANDAS["FS"].isnull() & cliquesPANDAS["FP"].isnull())
cliquesDERIVES=cliquesPANDAS[filtreDERIVE]

In [32]:
def fideliteLexique(numero,clique):
    formesLexicales=[]
    paradigmeClique={}
    formesPresentes=[]
    formesAbsentes=[]
    #####################
    #Prépararer la représentation lexicale
    #####################
    #
    # transformer les cases lexicales en points
    # au format clique pour comparaison
    #
    #####################

    #####################
    #Identifier le lexème
    #####################
    point=clique[0].split("-")
    lPoint=len(point)
    if lPoint==3:
        lexeme=point[0]
    else:
        lexeme="-".join(point[0:len(point)-2])
    paradigmeClique["lexeme"]=lexeme
    #####################
    #Nombre de cases lexicales
    #####################
    casesLexeme=casesLexicales[lexeme]
    nbCasesLexicales=len(casesLexeme)
    nbFormesLexicales=nbCasesLexicales
    #####################
    #Transformation des cases lexicales
    #####################
    if casesLexeme:
        for element in casesLexeme:
            champForme=paradigmes[paradigmes["lexeme"]==lexeme][element].iloc[0]
            if ","  in champForme:
                formes=champForme.split(",")
                nbFormesLexicales+=len(formes)-1
                for forme in formes:
                    pointCase=u"%s-%s-%s"% (lexeme,forme,element)
                    formesLexicales.append(pointCase)
                    if pointCase in clique:
                        formesPresentes.append(pointCase)
                    else:
                        formesAbsentes.append(pointCase)
            else:
                forme=champForme
                pointCase=u"%s-%s-%s"% (lexeme,forme,element)
                formesLexicales.append(pointCase)
                if pointCase in clique:
                    formesPresentes.append(pointCase)
                else:
                    formesAbsentes.append(pointCase)

    #######################################
    #
    # Compter les formes fidèles
    # Préparer le résultat
    #
    #######################################
    for element in clique:
        elements=element.split("-")
        forme=elements[-2]
        taminfo=elements[-1]
        paradigmeClique[taminfo]=forme
    paradigmeClique["score"]=cliquesScores[numero]
    paradigmeClique["ajouts"]=len(clique)-len(formesPresentes)
    paradigmeClique["lexOk"]=len(formesPresentes)
    paradigmeClique["lexMiss"]=len(formesAbsentes)
    return (paradigmeClique,formesPresentes,formesAbsentes,formesLexicales)

In [33]:
n=204
fideliteLexique(n,cliques[n])


({u'ai1S': u'akrwasi',
  u'ai3P': u'akrwasir',
  'ajouts': 11,
  u'ii1S': u'akrwasE',
  u'ii2S': u'akrwasE',
  u'ii3P': u'akrwasE',
  u'ii3S': u'akrwasE',
  u'inf': u'akrwasr',
  'lexMiss': 4,
  'lexOk': 1,
  'lexeme': u'accro\xeetre',
  u'pI1P': u'akrwas\xf4',
  u'pI2P': u'akrwase',
  u'pP': u'akrwas\xe2',
  u'pi2P': u'akrwase',
  u'pi3P': u'akrwas',
  'score': 13.94690794387567},
 [u'accro\xeetre-akrwasE-ii3S'],
 [u'accro\xeetre-akry-ppMS',
  u'accro\xeetre-akrwatr-inf',
  u'accro\xeetre-akrwa-pi3S',
  u'accro\xeetre-akry-ai3S'],
 [u'accro\xeetre-akry-ppMS',
  u'accro\xeetre-akrwasE-ii3S',
  u'accro\xeetre-akrwatr-inf',
  u'accro\xeetre-akrwa-pi3S',
  u'accro\xeetre-akry-ai3S'])

## Comparer la sortie des cliques avec le paradigme de départ

In [34]:
%%time
seuilClique=1
#paradigmesCLIQUES=pd.DataFrame(columns=paradigmes.columns.values.tolist())
paradigmesListe=[]
#for n,clique in enumerate(sorted(cliques,key=lambda x: len(x),reverse=True)):
progressBar = FloatProgress(min=0, max=len(cliques))
display(progressBar)
for n,clique in enumerate(cliques):
    progressBar.value=n
#    if seuilClique==0:
#        seuilClique=len(clique)-15
    if len(clique)>seuilClique:
        (paradigmeClique,formesPresentes,formesAbsentes,formesLexicales)=fideliteLexique(n,clique)
        paradigmesListe.append(paradigmeClique)
#paradigmesCLIQUES=pd.DataFrame(paradigmesListe,columns=GOLD.columns.values.tolist()+[u"score",u"ajouts"])
paradigmesCLIQUES=pd.DataFrame(paradigmesListe,columns=["lexeme"]+analyseCases+[u"score",u"ajouts",u"lexOk",u"lexMiss"])
#print (seuilClique)

CPU times: user 13min 43s, sys: 13.8 s, total: 13min 57s
Wall time: 14min 4s


In [ ]:
paradigmesCLIQUES

In [35]:
filtreCLIQUES=~(paradigmesCLIQUES["FS"].isnull() & paradigmesCLIQUES["FP"].isnull())
paradigmesFILTRES=paradigmesCLIQUES[filtreCLIQUES]

In [36]:
def maxColonne(table,listeGroupe,colonne):
    return table[table.groupby(listeGroupe)[colonne].transform(max)==table[colonne]]

In [37]:
paradigmesMaxLex=maxColonne(paradigmesFILTRES,["lexeme"],"lexOk")
paradigmesMaxScore=maxColonne(paradigmesMaxLex,["lexeme"],"score")
paradigmeMAX=paradigmesMaxScore
paradigmeMAX[["lexeme","FS","FP"]]

,lexeme,FS,FP
41,absoudre,absut,absut
98,accalmir,akalmire,akalmire
160,accroître,akrwat,akrwat
400,agonir,agOnit,agOnit
586,aller,vat,vat
900,arriver,arive,arive
1143,asseoir,aswat,aswat
1376,astreindre,astrêt,astrêt
1558,atteindre,atêt,atêt
1735,attendre,atât,atât


In [38]:
paradigmeSILVER=pd.merge(paradigmes,paradigmeMAX,on=["lexeme"])

In [39]:
filtreNouveauxDerivesSILVER=(paradigmeSILVER["FS_x"].isnull() & paradigmeSILVER["FP_x"].isnull())
filtreFsLexSILVER=(paradigmeSILVER["FS_x"].notnull() &
    (paradigmeSILVER["FS_x"]!=paradigmeSILVER["FS_y"]) 
    )
filtreFpLexSILVER=(paradigmeSILVER["FP_x"].notnull() &
    (paradigmeSILVER["FP_x"]!=paradigmeSILVER["FP_y"]) 
    )
paradigmeSILVER[(~filtreNouveauxDerivesSILVER)&(filtreFsLexSILVER)][["lexeme","FS_x","FP_x","FS_y","FP_y"]]
paradigmeSILVER[(~filtreNouveauxDerivesSILVER)&(filtreFpLexSILVER)][["lexeme","FS_x","FP_x","FS_y","FP_y"]]
paradigmeSILVER[filtreNouveauxDerivesSILVER][["lexeme","FS_y","FP_y"]]


,lexeme,FS_y,FP_y
0,absoudre,absut,absut
1,accalmir,akalmire,akalmire
2,accroître,akrwat,akrwat
3,agonir,agOnit,agOnit
4,aller,vat,vat
6,asseoir,aswat,aswat
7,astreindre,astrêt,astrêt
8,atteindre,atêt,atêt
10,avancer,avâse,avâse
11,battre,bat,bat


In [40]:
paradigmeSILVER[filtreNouveauxDerivesSILVER][["lexeme","FS_y","FP_y"]].to_csv(path_or_buf="2015-Data/"+sampleFile.replace(".csv","-NouveauxDerives.csv"),encoding="utf8",sep=";")

In [41]:
paradigmeSILVER[["lexeme","FS_x","FP_x","FS_y","FP_y"]].to_csv(path_or_buf="2015-Data/"+sampleFile.replace(".csv","-TousDerives.csv"),encoding="utf8",sep=";")

In [42]:
with open("2015-Data/"+sampleFile.replace(".csv","-TousDerives.pkl"), 'wb') as output:
   pickle.dump(paradigmeSILVER, output, pickle.HIGHEST_PROTOCOL)

In [ ]:
paradigmeSILVER.to_csv(path_or_buf="2015-Data/"+sampleFile.replace(".csv","-Derive.csv"),encoding="utf8",sep=";")
#GOLD.to_csv(path_or_buf="2015-Data/"+analysisPrefix+'-Gold.csv',encoding="utf8",sep=";")

In [ ]:
pd.reset_option('display.max_rows')

In [ ]:
derivesFS=set(paradigmesCLIQUES[~(paradigmesCLIQUES["FS"].isnull())]["FS"].unique())
derivesFP=set(paradigmesCLIQUES[~(paradigmesCLIQUES["FP"].isnull())]["FP"].unique())
derivesFSFP=derivesFS|derivesFP
for n,derive in enumerate(sorted(derivesFSFP)):
    print (n,derive)

In [ ]:
paradigmesCLIQUES[~(paradigmesCLIQUES["FP"].isnull())]